In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.transforms import ScaledTranslation

from datetime import datetime
import subprocess
import os

mplstylefile = "include/aps.mplstyle"

In [ ]:
#Face Colors
face_colors = ["#fbb4ae",
"#b3cde3",
"#ccebc5",
"#decbe4",
"#fed9a6",
"#ffffcc",
"#e5d8bd",
"#fddaec",
"#f2f2f2"]

#Edge Colors
edge_colors = ["#c7847f",
"#839cb1",
"#9bb994",
"#ac9ab2",
"#caa877",
"#cbcc9b",
"#b3a78d",
"#caa8ba",
"#bfbfbf"]

#Edge Colors2
edge_colors2 = ["#955753",
"#556e82",
"#6c8966",
"#7d6c82",
"#98794a",
"#9a9b6c",
"#837860",
"#99798a",
"#8f8f8f"]

name_to_face_color = {}
name_to_edge_color = {}
name_to_edge_color2 = {}

name_to_face_color["deac"] = face_colors[3]
name_to_edge_color["deac"] = edge_colors[3]
name_to_edge_color2["deac"] = edge_colors2[3]

name_to_face_color["fesom"] = face_colors[2]
name_to_edge_color["fesom"] = edge_colors[2]
name_to_edge_color2["fesom"] = edge_colors2[2]

name_to_face_color["maxent"] = face_colors[1]
name_to_edge_color["maxent"] = edge_colors[1]
name_to_edge_color2["maxent"] = edge_colors2[1]

ntfc = name_to_face_color
ntec = name_to_edge_color
ntec2 = name_to_edge_color2

# MAXENT cartoon figure

In [ ]:
data = np.load("../data/maxent_cartoon_figure.npz")

In [ ]:
with plt.style.context("include/aps.mplstyle"):
    fig, ax = plt.subplots(dpi=120, constrained_layout=True)
    ax.loglog(data["x"],data["y"],color=ntfc["maxent"])
    cx,cy,radius = data["circle_info"];

    # use the axis scale tform to figure out how far to translate 
    circ_offset = ScaledTranslation(10**cx,10**cy,ax.transScale)

    # construct the composite tform
    circ_tform = circ_offset + ax.transLimits + ax.transAxes

    # create the circle centred on the origin, apply the composite tform

    #circ = plt.Circle((10**cx,10**cy),radius=radius,color='r',alpha=0.5,transform=circ_tform)
    circ = plt.Circle((0,0),radius=radius,color=face_colors[0],ec=edge_colors[0],alpha=1.0,transform=circ_tform)

    ax.add_artist(circ)
    ax.set_aspect("equal")
    ax.set_ylabel(r"Log Likelihood $\chi^2$")
    ax.set_xlabel(r"Regularization Constant $\alpha$")
    t_default = ax.text(0.95, 0.95, 'default model', horizontalalignment='right', verticalalignment='top',
                        transform=ax.transAxes,
                        bbox=dict(
                            boxstyle="round",
                            ec="None",
                            fc="w",
                            alpha=0.5
                        ))
    t_information = ax.text(0.5, 0.55, 'information-fitting', horizontalalignment='center', verticalalignment='center',
                        transform=ax.transAxes,
                        bbox=dict(
                            boxstyle="round",
                            ec="None",
                            fc="w",
                            alpha=0.5
                        ))
    t_noise = ax.text(0.05, 0.05, 'noise-fitting', horizontalalignment='left', verticalalignment='bottom',
                        transform=ax.transAxes,
                        bbox=dict(
                            boxstyle="round",
                            ec="None",
                            fc="w",
                            alpha=0.5
                        ))
    #fig.savefig("../figures/maxent_cartoon.png",dpi=400)
    fig.savefig("../figures/maxent_cartoon.pdf")

# Nine panel plot

In [ ]:
deac_data = np.load("../data/deac_nomoments_spectra_smooth.npz")
fesom_data = np.load("../data/fesom_spectra_smooth.npz")
maxent_data = np.load("../data/maxent_spectra.npz")

In [ ]:
labels = [
    "shf",
    "shc",
    "sho",
    "stf",
    "stc",
    "sto",
    "tsf",
    "tsc",
    "tso",
    ]
error_labels = ["_large","_medium","_small"];

temperature = 1.2;
beta = 1/temperature;

mplstylefile = "include/aps.mplstyle"
#mplstylefile = "default"
for el in error_labels:
    with plt.style.context(mplstylefile):
        fig,axs = plt.subplots(dpi=240,figsize=(plt.rcParams['figure.figsize'][0]*2,plt.rcParams['figure.figsize'][1]*2),nrows=3,ncols=3,sharex=True,sharey=True,constrained_layout=True)
        for j,ax in enumerate(axs.flatten()):
            l = labels[j] + el;
            deac_frequency = deac_data[l][0,:]
            deac_dsf = deac_data[l][1,:]
            fesom_frequency = fesom_data[l][0,:]
            fesom_dsf = fesom_data[l][1,:]
            maxent_frequency = maxent_data[l][0,:]
            maxent_dsf = maxent_data[l][1,:]
            exact_frequency = deac_data[labels[j] + "_exact"][0,:]
            exact_dsf = deac_data[labels[j] + "_exact"][1,:]
            ax.plot(maxent_frequency,maxent_dsf,color=ntfc["maxent"],linewidth=0.5)
            ax.plot(fesom_frequency,fesom_dsf,color=ntfc["fesom"],linewidth=0.5)
            ax.plot(deac_frequency,deac_dsf,color=ntfc["deac"],linewidth=0.5)
            ax.plot(exact_frequency,exact_dsf,color="k",linestyle=":",linewidth=0.5)
            ax.text(0.05, 0.95, r"{}".format(labels[j]),
                  verticalalignment='top',
                  horizontalalignment='left',
                  transform=ax.transAxes,
                  color='black')
            ax.set_ylim((0,0.12))
        axs[2,2].plot([],[],color=ntfc["deac"],label=r"DEAC",linewidth=0.5)
        axs[2,2].plot([],[],color=ntfc["fesom"],label=r"FESOM",linewidth=0.5)
        axs[2,2].plot([],[],color=ntfc["maxent"],label=r"MEM",linewidth=0.5)
        axs[2,2].plot([],[],linestyle=":",color="k",label=r"Exact",linewidth=0.5)
        
        axs[2,2].legend(frameon=False,loc=0,handlelength=1.0)

        axs[0,0].set_xticks([0,16,32,48,64])
        axs[0,0].set_xticklabels([0,16,32,48,64])
        axs[2,1].set_xlabel(r"Frequency $\omega\ \mathrm{[K]}$")
        axs[1,0].set_ylabel(r"Dynamic Structure Factor $S(\mathbf{q},\omega)\ \mathrm{[K^{-1}]}$")
        fig.set_constrained_layout_pads(w_pad=1/72, h_pad=1/72, hspace=0, wspace=0)
        #fig.savefig("../figures/nine_panel" + el + ".png")
        fig.savefig("../figures/nine_panel" + el + ".pdf")

In [ ]:

temperature = 1.2;
beta = 1/temperature;
with plt.style.context(mplstylefile):
    fig,ax = plt.subplots(dpi=120,constrained_layout=True)
    l = "tso_medium";
    deac_frequency = deac_data[l][0,:]
    deac_dsf = deac_data[l][1,:]
    fesom_frequency = fesom_data[l][0,:]
    fesom_dsf = fesom_data[l][1,:]
    maxent_frequency = maxent_data[l][0,:]
    maxent_dsf = maxent_data[l][1,:]
    exact_frequency = deac_data["tso" + "_exact"][0,:]
    exact_dsf = deac_data["tso" + "_exact"][1,:]
    ax.plot(maxent_frequency,maxent_dsf,color=ntfc["maxent"])
    ax.plot(fesom_frequency,fesom_dsf,color=ntfc["fesom"])
    ax.plot(deac_frequency,deac_dsf,color=ntfc["deac"])
    ax.plot(exact_frequency,exact_dsf,color="k",linestyle=":")
    ax.text(0.95, 0.95, "tso medium",
          verticalalignment='top',
          horizontalalignment='right',
          transform=ax.transAxes,
          color='black')

    ax.text(0.05, 0.95, "$ϵ=0.001$",
          verticalalignment='top',
          horizontalalignment='left',
          transform=ax.transAxes,
          color='black')

    ax.plot([],[],color=ntfc["deac"],label=r"DEAC")
    ax.plot([],[],color=ntfc["fesom"],label=r"FESOM")
    ax.plot([],[],color=ntfc["maxent"],label=r"MEM")
    ax.plot([],[],linestyle=":",color="k",label=r"Exact")

    ax.legend(frameon=False,loc=4,handlelength=1.0)

    ax.set_xlim((0,48))
    ax.set_ylim((0,0.12))
    ax.set_xticks([0,16,32])
    ax.set_xticklabels([0,16,32])
    ax.set_xlabel(r"Frequency $\omega\ \mathrm{[K]}$")
    ax.set_ylabel(r"$S(\mathbf{q},\omega)\ \mathrm{[K^{-1}]}$")
    #fig.savefig("../figures/tso_medium.png")
    fig.savefig("../figures/tso_medium.pdf")

# Wallclock Time

In [ ]:
deac_data = np.load("../data/deac_nomoments_wallclock_time.npz")
fesom_data = np.load("../data/fesom_wallclock_time.npz")
maxent_data = np.load("../data/maxent_wallclock_time.npz")

In [ ]:
labels = [
    r"$\mathrm{same\ height\ far}$",
    r"$\mathrm{same\ height\ close}$",
    r"$\mathrm{same\ height\ overlapping}$",
    r"$\mathrm{short\ tall\ far}$",
    r"$\mathrm{short\ tall\ close}$",
    r"$\mathrm{short\ tall\ overlapping}$",
    r"$\mathrm{tall\ short\ far}$",
    r"$\mathrm{tall\ short\ close}$",
    r"$\mathrm{tall\ short\ overlapping}$",
    ];

labels = [
    "shf",
    "shc",
    "sho",
    "stf",
    "stc",
    "sto",
    "tsf",
    "tsc",
    "tso",
    ];

error_labels = ["_large","_medium","_small"];

temperature = 1.2;
beta = 1/temperature;

mplstylefile = "include/aps.mplstyle"
#mplstylefile = "default"

with plt.style.context(mplstylefile):

    width = 0.40  # the width of the bars
    x = np.arange(len(labels))*(width*3 + 0.2)  # the label locations

    deac_wallclock_large = np.zeros(len(labels))
    fesom_wallclock_large = np.zeros(len(labels))
    maxent_wallclock_large = np.zeros(len(labels))
    el = "_large"
    for i,ll in enumerate(labels):
        l = ll + el
        deac_wallclock_large[i] = deac_data[l]
        fesom_wallclock_large[i] = fesom_data[l]
        maxent_wallclock_large[i] = maxent_data[l]
    fig, ax = plt.subplots(constrained_layout=True)
    rects1 = ax.bar(x - width, maxent_wallclock_large, width, label=r'MEM',color=ntfc["maxent"], edgecolor=ntec["maxent"])
    rects2 = ax.bar(x, fesom_wallclock_large, width, label=r'FESOM',color=ntfc["fesom"], edgecolor=ntec["fesom"])
    rects3 = ax.bar(x + width, deac_wallclock_large, width, label=r'DEAC',color=ntfc["deac"], edgecolor=ntec["deac"])

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel(r"Wallclock Time $\mathrm{[hrs]}$")
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation = 45, ha="right")
    ax.legend(fontsize="small")


    # def autolabel(rects):
    #     """Attach a text label above each bar in *rects*, displaying its height."""
    #     for rect in rects:
    #         height = rect.get_height()
    #         ax.annotate('{}'.format(height),
    #                     xy=(rect.get_x() + rect.get_width() / 2, height),
    #                     xytext=(0, 3),  # 3 points vertical offset
    #                     textcoords="offset points",
    #                     ha='center', va='bottom')


    # autolabel(rects1)
    # autolabel(rects2)

    #fig.savefig("../figures/wallclock_time" + el + ".png")
    fig.savefig("../figures/wallclock_time" + el + ".pdf")

# CPU Time

In [ ]:
deac_data = np.load("../data/deac_nomoments_cpu_time.npz")
fesom_data = np.load("../data/fesom_cpu_time.npz")
maxent_data = np.load("../data/maxent_cpu_time.npz")

In [ ]:
labels = [
    r"$\mathrm{same\ height\ far}$",
    r"$\mathrm{same\ height\ close}$",
    r"$\mathrm{same\ height\ overlapping}$",
    r"$\mathrm{short\ tall\ far}$",
    r"$\mathrm{short\ tall\ close}$",
    r"$\mathrm{short\ tall\ overlapping}$",
    r"$\mathrm{tall\ short\ far}$",
    r"$\mathrm{tall\ short\ close}$",
    r"$\mathrm{tall\ short\ overlapping}$",
    ];

labels = [
    "shf",
    "shc",
    "sho",
    "stf",
    "stc",
    "sto",
    "tsf",
    "tsc",
    "tso",
    ];

error_labels = ["_large","_medium","_small"];

temperature = 1.2;
beta = 1/temperature;

mplstylefile = "include/aps.mplstyle"
#mplstylefile = "default"

time_data_table = np.zeros((9,9))
for i_el, el in enumerate(error_labels):
    #print(el)
    with plt.style.context(mplstylefile):

        width = 0.40  # the width of the bars
        x = np.arange(len(labels))*(width*3 + 0.2)  # the label locations

        deac_CPU_large = np.zeros(len(labels))
        fesom_CPU_large = np.zeros(len(labels))
        maxent_CPU_large = np.zeros(len(labels))
        #el = "_large"
        for i,ll in enumerate(labels):
            l = ll + el
            deac_CPU_large[i] = deac_data[l]
            fesom_CPU_large[i] = fesom_data[l]
            maxent_CPU_large[i] = maxent_data[l]
            time_data_table[i,i_el*3] = deac_data[l]
            time_data_table[i,i_el*3 + 1] = fesom_data[l]
            time_data_table[i,i_el*3 + 2] = maxent_data[l]
        #print(labels)
        #print("fesom")
        #print(fesom_CPU_large/deac_CPU_large)
        #print("maxent")
        #print(maxent_CPU_large/deac_CPU_large)
        fig, ax = plt.subplots(constrained_layout=True)
        rects1 = ax.bar(x - width, maxent_CPU_large, width, label=r'MEM',color=ntfc["maxent"], edgecolor=ntec["maxent"])
        rects2 = ax.bar(x, fesom_CPU_large, width, label=r'FESOM',color=ntfc["fesom"], edgecolor=ntec["fesom"])
        rects3 = ax.bar(x + width, deac_CPU_large, width, label=r'DEAC',color=ntfc["deac"], edgecolor=ntec["deac"])

        # Add some text for labels, title and custom x-axis tick labels, etc.
        ax.set_ylabel(r"CPU Time $\mathrm{[hrs]}$")
        ax.set_yscale("log")
        ax.set_xticks(x)
        ax.set_xticklabels(labels, rotation = 45, ha="right")
        ax.legend(fontsize="small",
                  frameon=True,
                  framealpha=0.5,
                  facecolor="white"
        )


        # def autolabel(rects):
        #     """Attach a text label above each bar in *rects*, displaying its height."""
        #     for rect in rects:
        #         height = rect.get_height()
        #         ax.annotate('{}'.format(height),
        #                     xy=(rect.get_x() + rect.get_width() / 2, height),
        #                     xytext=(0, 3),  # 3 points vertical offset
        #                     textcoords="offset points",
        #                     ha='center', va='bottom')


        # autolabel(rects1)
        # autolabel(rects2)
        fig.savefig("../figures/CPU_time" + el + ".png")
        fig.savefig("../figures/CPU_time" + el + ".pdf")

In [ ]:
print("small")
print(np.mean(time_data_table[:,1]/time_data_table[:,0]))
print(np.mean(time_data_table[:,2]/time_data_table[:,0]))

print("medium")
print(np.mean(time_data_table[:,1 + 3]/time_data_table[:,0 + 3]))
print(np.mean(time_data_table[:,2 + 3]/time_data_table[:,0 + 3]))

print("large")
_tF = np.mean(time_data_table[:,1 + 6]/time_data_table[:,0 + 6])
_tM = np.mean(time_data_table[:,2 + 6]/time_data_table[:,0 + 6])
print(_tF)
print(_tM)
print((_tF + _tM)/2)


# Goodness of fit

In [ ]:
deac_data = np.load("../data/deac_nomoments_lack_of_fit_smooth.npz")
fesom_data = np.load("../data/fesom_lack_of_fit_smooth.npz")
maxent_data = np.load("../data/maxent_lack_of_fit.npz")

In [ ]:
labels = [
    r"$\mathrm{same\ height\ far}$",
    r"$\mathrm{same\ height\ close}$",
    r"$\mathrm{same\ height\ overlapping}$",
    r"$\mathrm{short\ tall\ far}$",
    r"$\mathrm{short\ tall\ close}$",
    r"$\mathrm{short\ tall\ overlapping}$",
    r"$\mathrm{tall\ short\ far}$",
    r"$\mathrm{tall\ short\ close}$",
    r"$\mathrm{tall\ short\ overlapping}$",
    ];

labels = [
    "shf",
    "shc",
    "sho",
    "stf",
    "stc",
    "sto",
    "tsf",
    "tsc",
    "tso",
    ];

error_labels = ["_small","_medium","_large"];

temperature = 1.2;
beta = 1/temperature;

#mplstylefile = "default"

data_table = np.zeros((9,9))
for i_el, el in enumerate(error_labels):
    #print(el)
    with plt.style.context(mplstylefile):

        width = 0.40  # the width of the bars
        x = np.arange(len(labels))*(width*3 + 0.2)  # the label locations

        deac_lof_large = np.zeros(len(labels))
        fesom_lof_large = np.zeros(len(labels))
        maxent_lof_large = np.zeros(len(labels))
        #el = "_small"
        for i,ll in enumerate(labels):
            l = ll + el
            deac_lof_large[i] = deac_data[l]
            fesom_lof_large[i] = fesom_data[l]
            maxent_lof_large[i] = maxent_data[l]
            data_table[i,i_el*3] = deac_data[l]
            data_table[i,i_el*3 + 1] = fesom_data[l]
            data_table[i,i_el*3 + 2] = maxent_data[l]
        #print(labels)
        #print("fesom")
        #print(fesom_lof_large/deac_lof_large)
        #print("maxent")
        #print(maxent_lof_large/deac_lof_large)
        #print(maxent_lof_large)
        fig, ax = plt.subplots(constrained_layout=True)
        rects1 = ax.bar(x - width, maxent_lof_large, width, label=r'MEM',color=ntfc["maxent"], edgecolor=ntec["maxent"])
        rects2 = ax.bar(x, fesom_lof_large, width, label=r'FESOM',color=ntfc["fesom"], edgecolor=ntec["fesom"])
        rects3 = ax.bar(x + width, deac_lof_large, width, label=r'DEAC',color=ntfc["deac"], edgecolor=ntec["deac"])

        # Add some text for labels, title and custom x-axis tick labels, etc.
        ax.set_ylabel("Goodness of Fit $\\varphi_\mathrm{lof}$")

        ax.set_xticks(x)
        ax.set_xticklabels(labels, rotation = 45, ha="right")
        ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
        ax.legend(fontsize="small")


        # def autolabel(rects):
        #     """Attach a text label above each bar in *rects*, displaying its height."""
        #     for rect in rects:
        #         height = rect.get_height()
        #         ax.annotate('{}'.format(height),
        #                     xy=(rect.get_x() + rect.get_width() / 2, height),
        #                     xytext=(0, 3),  # 3 points vertical offset
        #                     textcoords="offset points",
        #                     ha='center', va='bottom')


        # autolabel(rects1)
        # autolabel(rects2)

        fig.savefig("../figures/lof" + el + ".png")
        fig.savefig("../figures/lof" + el + ".pdf")

In [ ]:
for i in range(9):
    s_fmt = [labels[i]]
    for j in range(9):
        #s = "${:.2e}$".format(data_table[i,j])
        #s = " \\times 10^".join(s.split("e"))
        s = "{:.3f}".format(np.log10(data_table[i,j]))
        
        s_fmt.append(s)
    print(" | ".join(s_fmt))

In [ ]:
print(np.mean(np.log10(data_table[:,0]) - np.log10(data_table[:,1])))
print(np.mean(np.log10(data_table[:,0]) - np.log10(data_table[:,2])))

print(np.mean(np.log10(data_table[:,0 + 3]) - np.log10(data_table[:,1 + 3])))
print(np.mean(np.log10(data_table[:,0 + 3]) - np.log10(data_table[:,2 + 3])))

print(np.mean(np.log10(data_table[:,0 + 6]) - np.log10(data_table[:,1 + 6])))
print(np.mean(np.log10(data_table[:,0 + 6]) - np.log10(data_table[:,2 + 6])))


In [ ]:
gof_improvement_fesom = np.zeros(3)
gof_improvement_mem = np.zeros(3)

gof_improvement_fesom[0] = np.mean(np.log10(data_table[:,0]) - np.log10(data_table[:,1]))
gof_improvement_mem[0] = np.mean(np.log10(data_table[:,0]) - np.log10(data_table[:,2]))

gof_improvement_fesom[1] = np.mean(np.log10(data_table[:,0 + 3]) - np.log10(data_table[:,1 + 3]))
gof_improvement_mem[1] = np.mean(np.log10(data_table[:,0 + 3]) - np.log10(data_table[:,2 + 3]))

gof_improvement_fesom[2] = np.mean(np.log10(data_table[:,0 + 6]) - np.log10(data_table[:,1 + 6]))
gof_improvement_mem[2] = np.mean(np.log10(data_table[:,0 + 6]) - np.log10(data_table[:,2 + 6]))

print("small")
print(gof_improvement_fesom[0])
print(gof_improvement_mem[0])

print("medium")
print(gof_improvement_fesom[1])
print(gof_improvement_mem[1])

print("large")
print(gof_improvement_fesom[2])
print(gof_improvement_mem[2])

In [ ]:
def gof_improvement2(x):
    return 1/(10**x)
print("small")
print(gof_improvement2(gof_improvement_fesom[0]))
print(gof_improvement2(gof_improvement_mem[0]))

print("medium")
print(gof_improvement2(gof_improvement_fesom[1]))
print(gof_improvement2(gof_improvement_mem[1]))

print("large")
print(gof_improvement2(gof_improvement_fesom[2]))
print(gof_improvement2(gof_improvement_mem[2]))

In [ ]:
data_table[:,1]/data_table[:,0]

In [ ]:
print(r"\begin{table*}[t]")
print(r"    \renewcommand{\arraystretch}{1.5}")
print(r"    \setlength\tabcolsep{12pt}")
print(r"    \begin{tabular}{@{}lc@{  }c@{  }cc@{  }c@{  }cc@{  }c@{  }c@{  }}")
print(r"        \toprule")
print(r"        & \multicolumn{3}{c}{DEAC} & \multicolumn{3}{c}{FESOM} & \multicolumn{3}{c}{MEM} \\")
print(r"        Alias & s & m & l & s & m & l & s & m & l \\")
print(r"        \midrule")
for i in range(9):
    s_fmt = [labels[i]]
    for j in range(9):
        #s = "${:.2e}$".format(data_table[i,j])
        #s = " \\times 10^".join(s.split("e"))
        s = "${:.3f}$".format(np.log10(data_table[i,j]))
        
        s_fmt.append(s)
    print("        " + " & ".join(s_fmt) + r" \\")
print(r"        \bottomrule")
print(r"    \end{tabular}")
print(r"    \caption{\label{tab:goodness_of_fit}")
print(r"        FIXME")
print(r"    }")
print(r"\end{table*}")

# Helium Dispersion Peaks

In [ ]:
helium_dispersion_peaks_data = np.load("../data/helium_dispersion_peaks.npz")

wavevectors = helium_dispersion_peaks_data["wavevectors"]
dispersion = helium_dispersion_peaks_data["dispersion"]
momenta = helium_dispersion_peaks_data["momenta"]
peak_locations_mean = helium_dispersion_peaks_data["peak_locations_mean"]
peak_locations_error = helium_dispersion_peaks_data["peak_locations_error"]

In [ ]:
with plt.style.context(mplstylefile):
    fig,ax = plt.subplots(dpi=240,constrained_layout=True)
    ax.plot(wavevectors,dispersion,color="r",label="Experiment")
    ax.errorbar(momenta,peak_locations_mean,peak_locations_error,marker="o",linestyle="None",label="QMC + DEAC",color=name_to_edge_color["deac"])
    #ax.set_yscale("log")
    ax.set_ylim((0,2.0))
    ax.set_xlim((0,2.9))
    ax.set_ylabel("Energy Transfer $\\omega\\ \\mathrm{meV}$",fontweight="regular")
    ax.set_xlabel("Wavevector $q\ [\mathrm{Å^{-1}}]$",fontweight="regular")
    ax.legend()
    fig.savefig("../figures/helium_dispersion.pdf")

# Bulk Spectrum

In [ ]:
helium_dispersion_peaks_data = np.load("../data/helium_dispersion_peaks.npz")

wavevectors = helium_dispersion_peaks_data["wavevectors"]
dispersion = helium_dispersion_peaks_data["dispersion"]

dsf_bulk_data = np.load("../data/bulk_helium_spectrum_smooth.npz")
dsf_bulk = dsf_bulk_data["dsf"]
extent_bulk = dsf_bulk_data["extent"]

In [ ]:
from scipy.constants import physical_constants as spc

In [ ]:
K_to_meV = spc["Boltzmann constant in eV/K"][0]*1000

In [ ]:
with plt.style.context(mplstylefile):
    fig,ax = plt.subplots(dpi=240,constrained_layout=True)
    ylim = (extent_bulk[2], extent_bulk[3])
    ylim = (0,4)
    aspect = (extent_bulk[1] - extent_bulk[0])/(ylim[1] - ylim[0]);
    aspect = plt.rcParams['figure.figsize'][1]/plt.rcParams['figure.figsize'][0]/1.1
    im = ax.imshow(dsf_bulk/K_to_meV/(2*np.pi),origin="lower",extent=extent_bulk,aspect=aspect,interpolation="None")
    ax.plot(wavevectors,dispersion,linestyle=":",color="r")
    ax.set_ylabel("Energy Transfer $\\omega\\ \\mathrm{[meV]}$",fontweight="regular")
    ax.set_xlabel("Wavevector $q\\ [\\mathrm{Å^{-1}}]$",fontweight="regular")
    ax.set_ylim(ylim)
    fig.colorbar(im,label="$S(q,\omega)\\  \\mathrm{[meV^{-1}]}$",extend="max")
    fig.savefig("../figures/bulk_he_spectrum.pdf")